In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Frame the problem
Using the customer description, Define the problem your trying to solve in your own words (remember this is not technial but must be specific so the customer understands the project(

Given a prebuilt Tetris simulator built using Pygame, design a machine learning model, namely a genetic learning model that will play the game at a reasonable pace for a minimum of 60 seconds. 

# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

There is no initial data. Any data gathered is gathered during training where the model is told to improve after being told to play the game and learning from there. This is because the way the model is being trained is through constant "breeding" where the best of a generation are allowed to survive and breed until the next game while those that aren't die. The initial model is extremely unintelligent as it has no input data and hasn't learned anything from playing the game.  

# 3. Explore the Data
Gain insights into the data you have from step 2, making sure to identify any bias

There is 7 distinct tetrominos, where single set of free tetrominoes or one-sided tetrominoes cannot fit in a rectangle and  any odd number of sets for either type cannot fit in a rectangle

Just by looking at it, Genetic seems to be quite poor, while greedy is quite a good model. MCTS also seems quite lacking, most likely as these are framesworks for a model, and not actual models. 


In [ ]:
!python libraries/main.py

In [1]:
!python libraries/main.py genetic

pygame 2.5.2 (SDL 2.28.2, Python 3.10.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
0
14 0
1
8 0
Traceback (most recent call last):
  File "/home/jupyter-michaelw/Project_5/libraries/main.py", line 16, in <module>
    main()
  File "/home/jupyter-michaelw/Project_5/libraries/main.py", line 11, in main
    g.run_no_visual()
  File "/home/jupyter-michaelw/Project_5/libraries/game.py", line 41, in run_no_visual
    x, piece = self.ai.get_best_move(self.board, self.curr_piece)
  File "/home/jupyter-michaelw/Project_5/libraries/AI/genetic.py", line 82, in get_best_move
    board_copy[y + pos[1]][x + pos[0]] = True
IndexError: list index out of range


In [2]:
!python libraries/main.py greedy

pygame 2.5.2 (SDL 2.28.2, Python 3.10.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
0
1091 439
1
730 293
731 293
AI
Traceback (most recent call last):
  File "/home/jupyter-michaelw/Project_5/libraries/main.py", line 16, in <module>
    main()
  File "/home/jupyter-michaelw/Project_5/libraries/main.py", line 12, in main
    g.run()
  File "/home/jupyter-michaelw/Project_5/libraries/game.py", line 72, in run
    x, piece = self.ai.get_best_move(self.board, self.curr_piece)
  File "/home/jupyter-michaelw/Project_5/libraries/AI/greedy.py", line 32, in get_best_move
    c = self.cost(board.board, x, y, piece)
  File "/home/jupyter-michaelw/Project_5/libraries/AI/greedy.py", line 97, in cost
    board_copy[y + pos[1]][x + pos[0]] = True
IndexError: list index out of range


In [1]:
!python libraries/main.py mcts

pygame 2.5.2 (SDL 2.28.2, Python 3.10.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
0
9 0
1
8 0
9 0
AI
Pieces dropped: 10
Rows cleared: 0


# 4.Prepare the Data

No data transformations, everything that needs to be done to the input data has already been prepared by Mr. Rivero in game.py


# 5. Model the data
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a Model to continue with


I made a basic neural network, as it is probably the one way to properly predict an output for a given input file. I did two methods, one using a prebuilt model using SKlean and MLPClassifier, and one by trying to build one with keras. Both are extremely bad, which is likely due to an issue with the data itself. I will finetune it just to make sure if it can be salvaged. 

In [ ]:
from versionHistory.v1 import v1
from multiprocessing import Pool
from libraries.game import Game
import random
import numpy as np
import joblib

random.seed()

#population size
populationSize = 1000
#amount of generations
genLimit = 10000
population = [v1() for _ in range(populationSize)]

def runGame(i):
    game = Game(mode="tetrisAI", agent=i)
    dropped, cleared = game.run_no_visual()
    #print(f'Dropped: {dropped}, Cleared: {cleared}')
    return cleared

def breed(parent1, parent2):
    if parent1 == None or parent2 == None:
        return None
    max_len = max(len(parent1.genotype), len(parent2.genotype))
    
    # Pad the genotypes to the maximum length
    padded_parent1 = np.pad(parent1.genotype, (0, max_len - len(parent1.genotype)))
    padded_parent2 = np.pad(parent2.genotype, (0, max_len - len(parent2.genotype)))
    
    # Ensure crossover_point is within the bounds of the shorter genotype
    crossover_point = random.randint(0, max_len)
    
    # Perform crossover with padded genotypes
    child_genotype = np.concatenate((padded_parent1[:crossover_point], padded_parent2[crossover_point:]))
    
    child = v1(genotype=child_genotype)
    return child

valVal, best = 5, 0
print('[', end='')
with open(f'v1/{valVal}/README.txt', 'w') as f:
    pass
    
with Pool(30) as pool:
    for gen in range(genLimit):
        scoreList = pool.map(runGame, population)
        listedScore = [(score, i) for i, score in enumerate(scoreList)]
        sortedScores = sorted(listedScore, key=lambda x: x[0], reverse=True)

        liversList = [i for _, i in sortedScores[:int(0.3*populationSize)]]
        #print(liversList)
        livers = [population[i] for i in liversList]

        genBest, childID = sortedScores[0][0], sortedScores[0][1]

        if genBest > best:
            best = genBest
            genNum = gen+1
            bestChildID = childID
            currentBest = population[int(childID)]
            currentBestScore = genBest

        if (gen+1)%1 == 0:
            print('.', end='')
            best = 0
            spacer = str(genLimit)[len(str(genNum+1)):]
            joblib.dump(currentBest, f'v1/{valVal}/v1g{spacer}{genNum}.joblib')
            with open(f'v1/{valVal}/README.txt', 'a') as f:
                f.write(f'Gen: {genNum}, Child {bestChildID} achieved score {currentBestScore}\n')
        if (gen+1)%1000 == 0:
            print(f']\nGen: {gen+1}, Child {childID} achieved score {genBest}\n[')
            #joblib.dump(population[int(childID)], f'v1/v1m{gen+1}.joblib')
            
        
        newPopulation = []
        _ = 0
        while _ < populationSize:
            parent1 = random.choice(livers)
            parent2 = random.choice(livers)
            child = breed(parent1, parent2)
            if child is not None:
                newPopulation.append(child)
                _ += 1

        population = newPopulation

    import joblib
    #joblib.dump(population[int(childID)], 'v1/v1m1.joblib')

print('Done')

pygame 2.5.2 (SDL 2.28.2, Python 3.10.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
[

# 6. Fine Tune the Model
Given that this model is a genetic learning algorithmn, there's not much that can be done in order to properly train it. Thus, I can only cycle through my trained models and find the best model from there.


In [ ]:
from versionHistory.v1 import v1
from multiprocessing import Pool
from libraries.game import Game
import random
import numpy as np
import joblib

with open('xdd.txt', 'w') as f:
    pass
def runGame(n):
    i = joblib.load(f'v1/5/v1g000{n}.joblib')
    game = Game(mode="tetrisAI", agent=i)
    dropped, cleared = game.run_no_visual()
    with open('xdd.txt', 'a') as f:
        f.write(f'{n}: Dropped: {dropped}, Cleared: {cleared}\n')
    

pop = [n for n in range(10, 32)]

with Pool(30) as pool:
    print('Starting')
    xdd = pool.map(runGame, pop)
    print('Done')
#runGame()

Starting


# 7. Present
In a customer faceing Document provide summery of finding and detail approach taken


1. Upon Recieving the Data, it was immediately inspected by running the game program and the various algorithmns that were given to me.
2. The program was then trained using a population of 1000 over the approximation of 30 generations. 

# 8. Launch the Model System
Define your production run code, This should be self susficent and require only your model pramaters 

See deployment.py (copied here anyways)

In [ ]:
from versionHistory.v1 import v1
from multiprocessing import Pool
from libraries.game import Game
import random
import numpy as np
import joblib

#11 is the best
def runGame():
    i = joblib.load('v1/5/v1g00011.joblib')
    game = Game(mode="tetrisAI", agent=i)
    dropped, cleared = game.run()
    #print(f'Dropped: {dropped}, Cleared: {cleared}')

runGame()